In [ ]:
from flask import Flask, render_template, request, session , redirect
import pyrebase
from tensorflow.keras.preprocessing import image
import numpy as np
import json
from tensorflow.keras.models import load_model
import base64
import csv
import requests

app = Flask(__name__)
app.secret_key = "your_secret_key"
my_model = load_model('model_trained.h5', compile=False)

class ImageRecognition:
    def predict_top_classes(model, images, top=3):  #
        predictions = []
        for img_path in images:
            img = image.load_img(img_path, target_size=(299, 299))
            img = image.img_to_array(img)
            img = np.expand_dims(img, axis=0)
            img /= 255.

            pred = model.predict(img)
            top_indices = pred.argsort()[0][-top:][::-1]
            top_predictions = [(food_list[i], pred[0][i]) for i in top_indices]
            predictions.append(top_predictions)
        return predictions

    @app.route('/predict', methods=['POST'])       
    def predict():
        uploaded_file = request.files['file']
        session['uploaded_filename'] = uploaded_file.filename
        uploaded_file.save(uploaded_file.filename)
        images = [uploaded_file.filename]
        top_predictions = predict_top_classes(my_model, images)[0]
        with open(uploaded_file.filename, "rb") as img_file:
            encoded_image = base64.b64encode(img_file.read()).decode('utf-8')
        return render_template('result.html', top_predictions=top_predictions, image=encoded_image)

# Define a function to read food names from a CSV file
def load_food_names_from_csv(csv_file):
    food_names = []
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            food_names.extend(row)
    return food_names

class uploadImage:
    @app.route('/upload')
    def uploadFile():
        return render_template('upload.html')
